<a href="https://colab.research.google.com/github/monch1962/training_and_deploying_an_ML_model_as_a_microservice/blob/master/score_reviews_via_service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1: Existing Machine Learning Services

<a href="https://colab.research.google.com/github/peckjon/hosting-ml-as-microservice/blob/master/part1/score_reviews_via_service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Obtain labelled reviews

In order to test any of the sentiment analysis APIs, we need a labelled dataset of reviews and their sentiment polarity. We'll use NLTK to download the movie_reviews corpus.

In [25]:
from nltk import download

download('movie_reviews')

[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


True

### Load the data

The files in movie_reviews have already been divided into two sets: positive ('pos') and negative ('neg'), so we can load the raw text of the reviews into two lists, one for each polarity.

In [0]:
from nltk.corpus import movie_reviews

# extract words from reviews, pair with label

reviews_pos = []
for fileid in movie_reviews.fileids('pos'):
    review = movie_reviews.raw(fileid)
    reviews_pos.append(review)

reviews_neg = []
for fileid in movie_reviews.fileids('neg'):
    review = movie_reviews.raw(fileid)
    reviews_neg.append(review)

### Connect to the scoring API

Fill in this function with code that connects to one of these APIs, and uses it to score a single review:

* [Amazon Comprehend: Detect Sentiment](https://docs.aws.amazon.com/comprehend/latest/dg/API_DetectSentiment.html)
* [Google Natural Language: Analyzing Sentiment](https://cloud.google.com/natural-language/docs/analyzing-sentiment)
* [Azure Cognitive Services: Sentiment Analysis](https://docs.microsoft.com/en-us/azure/cognitive-services/text-analytics/how-tos/text-analytics-how-to-sentiment-analysis)
* [Algorithmia: Sentiment Analysis](https://algorithmia.com/algorithms/nlp/SentimentAnalysis)

Your function must return either 'pos' or 'neg', so you'll need to make some decisions about how to map the results of the API call to one of these values. For example, Amazon Comprehend can return "NEUTRAL" or "MIXED" for the Sentiment -- if this happens, you may with to inspect the numeric values under the SentimentScore to see whether it leans toward positive or negative.


In [27]:
!pip install -U -q PyDrive
# Imports
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Google drive authentication
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# File params
local_save_dir = "/root/.aws"
filename = "credentials"
save_path = "{0}/{1}".format(local_save_dir, filename)

# Choose/create a local (colab) directory to store the data.
local_download_path = os.path.expanduser(local_save_dir)
try:
  os.makedirs(local_download_path)
except: pass

drive_list = drive.ListFile().GetList()
f = [x for x in drive_list if x["title"] == filename][0]

print('title: %s, id: %s' % (f['title'], f['id']))
fname = os.path.join(local_download_path, f['title'])
print('downloading to {}'.format(fname))
f_ = drive.CreateFile({'id': f['id']})
f_.GetContentFile(fname)

with open(save_path) as creds:
    for i, line in enumerate(creds):
        if i == 1:
            access_token_key = line.replace("aws_access_key_id=", "").replace("\n", "")
        if i == 2:
            access_token_secret = line.replace("aws_secret_access_key=", "").replace("\n", "")

title: credentials, id: 13qj2P7nEJSfKLO7tHDg107-uMFghVf9v
downloading to /root/.aws/credentials


In [28]:
import boto3
import json

def score_review(review):
    # TBD: call the service and return 'pos' or 'neg'
    comprehend = boto3.client(service_name='comprehend', region_name='ap-southeast-2')
    response = comprehend.detect_sentiment(Text=review[:5000], LanguageCode='en') # Workaround for max 5000 character limit inside AWS Comprehend
 
    if response["Sentiment"] == "POSITIVE":
        return 'pos'
    elif response["Sentiment"] == "NEGATIVE":
        return 'neg'
    elif response['SentimentScore']['Negative'] > response['SentimentScore']['Positive']:
        return 'neg'
    
    return 'pos'

Calling DetectSentiment
{
    "ResponseMetadata": {
        "HTTPHeaders": {
            "content-length": "167",
            "content-type": "application/x-amz-json-1.1",
            "date": "Wed, 27 May 2020 06:06:18 GMT",
            "x-amzn-requestid": "5bd64146-a6a5-4fc5-9c50-b6572f223db0"
        },
        "HTTPStatusCode": 200,
        "RequestId": "5bd64146-a6a5-4fc5-9c50-b6572f223db0",
        "RetryAttempts": 0
    },
    "Sentiment": "POSITIVE",
    "SentimentScore": {
        "Mixed": 2.2560412617167458e-05,
        "Negative": 0.0002988109190482646,
        "Neutral": 0.0012165051884949207,
        "Positive": 0.9984622001647949
    }
}
POSITIVE
End of DetectSentiment



### Score each review

Now, we can use the function you defined to score each of the reviews

In [0]:
results_pos = []
for review in reviews_pos:
    result = score_review(review)
    results_pos.append(result)

results_neg = []
for review in reviews_neg:
    result = score_review(review)
    results_neg.append(result)

### Calculate accuracy

For each of our known positive reviews, we can count the number which our function scored as 'pos', and use this to calculate the % accuracy. We repeaty this for negative reviews, and also for overall accuracy.

In [30]:
correct_pos = results_pos.count('pos')
accuracy_pos = float(correct_pos) / len(results_pos)
correct_neg = results_neg.count('neg')
accuracy_neg = float(correct_neg) / len(results_neg)
correct_all = correct_pos + correct_neg
accuracy_all = float(correct_all) / (len(results_pos)+len(results_neg))

print('Positive reviews: {}% correct'.format(accuracy_pos*100))
print('Negative reviews: {}% correct'.format(accuracy_neg*100))
print('Overall accuracy: {}% correct'.format(accuracy_all*100))

Positive reviews: 75.3% correct
Negative reviews: 56.99999999999999% correct
Overall accuracy: 66.14999999999999% correct
